In [22]:
import pandas as pd
import s3fs

In [23]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})
MY_BUCKET = "williamolivier"

FILE_PATH_S3_FVDEP = f"{MY_BUCKET}/diffusion/epci_features_7.csv"
with fs.open(FILE_PATH_S3_FVDEP, "rb") as file:
    epci_features_7 = pd.read_csv(file)

FILE_PATH_S3_FVDEP = f"{MY_BUCKET}/diffusion/epci_features_6.csv"
with fs.open(FILE_PATH_S3_FVDEP, "rb") as file:
    epci_features_6 = pd.read_csv(file)

FILE_PATH_S3_FVDEP = f"{MY_BUCKET}/diffusion/epci_features_5.csv"
with fs.open(FILE_PATH_S3_FVDEP, "rb") as file:
    epci_features_5 = pd.read_csv(file)

FILE_PATH_S3_FVDEP = f"{MY_BUCKET}/diffusion/epci_features_4.csv"
with fs.open(FILE_PATH_S3_FVDEP, "rb") as file:
    epci_features_4 = pd.read_csv(file)

étape 1 : Il faut choisir un clustering parmi les 4 (on demandera l'avis de Nicolas) et pour chaque cluster trouver une composition moyenne des transports en communs utilisé (ex 80% metro et 10% train et 10% bus pour le cluster ou y'a que paris, etc)

étape 2 : Il faut écrire un dictionnaire qui recense pour chaque type de transport en commun son émission moy/km 

étape 3 : Créer une nouvelle colonne sur un des 4 tableaux (selon le clustering retenu) qui donnerait une émission moyenne pour chaque EPCI (soit global, soit par type de flux (séparer rouge et jaune? à voir))

-> pour cette étape voir si + pertinent de prendre les flux entrant ou sortant ? ou une moyenne des 2 ?

étape 4 : trouver un moyen de repérer les mauvais élèves (par exemple ceux dont l'émission associée calculée est aberrante ou plus élevée par rapport aux autres EPCI du même cluster)

-> pour cette étape trouver un indicateurs pour détecter les mauvais élève (ex arbitraire 50% aus dessus de la moyenne) mais voir si on peut trouver un meilleur indicateur bien justifier

In [24]:
# Etape 1

# Pour l'instant choix du clustering à 7 clusters
# On prend en compte à la fois les flux entrants et les flux sortants pour faire les calculs

epci_features=epci_features_7 # il suffira de changer le numéro du clustering choisi ici

epci_features["nombre_total_de_flux_par_epci"]=(
    epci_features["nb_entrant_>50km_rouge"]
    + epci_features["nb_sortant_>50km_rouge"]
    + epci_features["nb_intra_rouge"]
    + epci_features["nb_entrant_>50km_vert"]
    + epci_features["nb_sortant_>50km_vert"]
    + epci_features["nb_intra_vert"]
    + epci_features["nb_entrant_>50km_jaune"]
    + epci_features["nb_sortant_>50km_jaune"]
    + epci_features["nb_intra_jaune"]
    )

# Calcul de la proportion d'individus qui se rendent au travail en voiture
epci_features["proportion_individus_flux_rouges"] = (
    (epci_features["nb_entrant_>50km_rouge"]
    + epci_features["nb_sortant_>50km_rouge"]
    + epci_features["nb_intra_rouge"])
    /epci_features["nombre_total_de_flux_par_epci"]
    )

# Calcul de la proportion d'individus qui se rendent au travail en transports en commun
epci_features["proportion_individus_flux_jaunes"] = (
    (epci_features["nb_entrant_>50km_jaune"]
    + epci_features["nb_sortant_>50km_jaune"]
    + epci_features["nb_intra_jaune"])
    /epci_features["nombre_total_de_flux_par_epci"]
    )

# Calcul de la proportion d'individus qui se rendent au travail à pieds / en vélo / qui télétravaillent ou travaillent à domicile
epci_features["proportion_individus_flux_verts"] = (
    (epci_features["nb_entrant_>50km_vert"]
    + epci_features["nb_sortant_>50km_vert"]
    + epci_features["nb_intra_vert"])
    /epci_features["nombre_total_de_flux_par_epci"]
    )

# print(epci_features["proportion_individus_flux_rouges"])
# les résultats sont parfaitement logiques ici

In [25]:
# Même chose avec la distance

epci_features["distance_totale_de_flux_par_epci"]=(
    epci_features["distance_entrant_>50km_rouge"]
    + epci_features["distance_sortant_>50km_rouge"]
    + epci_features["distance_intra_rouge"]
    + epci_features["distance_entrant_>50km_vert"]
    + epci_features["distance_sortant_>50km_vert"]
    + epci_features["distance_intra_vert"]
    + epci_features["distance_entrant_>50km_jaune"]
    + epci_features["distance_sortant_>50km_jaune"]
    + epci_features["distance_intra_jaune"]
    )

# Calcul de la proportion de kilomètres qui sont parcourus en voiture
epci_features["proportion_distance_flux_rouges"] = (
    (epci_features["distance_entrant_>50km_rouge"]
    + epci_features["distance_sortant_>50km_rouge"]
    + epci_features["distance_intra_rouge"])
    /epci_features["distance_totale_de_flux_par_epci"]
    )

# Calcul de la proportion de kilomètres qui sont parcourus en transports en commun
epci_features["proportion_distance_flux_jaunes"] = (
    (epci_features["distance_entrant_>50km_jaune"]
    + epci_features["distance_sortant_>50km_jaune"]
    + epci_features["distance_intra_jaune"])
    /epci_features["distance_totale_de_flux_par_epci"]
    )

# Calcul de la proportion de kilomètres qui sont parcourus à pieds / en vélo / qui télétravaillent ou travaillent à domicile
epci_features["proportion_distance_flux_verts"] = (
    (epci_features["distance_entrant_>50km_vert"]
    + epci_features["distance_sortant_>50km_vert"]
    + epci_features["distance_intra_vert"])
    /epci_features["distance_totale_de_flux_par_epci"]
    )

print(epci_features["proportion_distance_flux_rouges"])
print(epci_features["proportion_distance_flux_jaunes"])
print(epci_features["proportion_distance_flux_verts"])
print(epci_features["proportion_distance_flux_rouges"]+epci_features["proportion_distance_flux_jaunes"]+epci_features["proportion_distance_flux_verts"])
# La somme de toutes les proportions fait 1
# par contre c'est bizarre pcq la proportion de gesn qui vont au taf en voiture c'est 70% en moyenne
# mais la proportion de la distance parcourue en voiture c'est que 30% en moyenne
# pourtant je ne vois pas d'erreur dans les calculs

0       0.228653
1       0.250102
2       0.302975
3       0.343886
4       0.342399
          ...   
1220    0.362942
1221    0.329808
1222    0.337226
1223    0.298209
1224    0.235933
Name: proportion_distance_flux_rouges, Length: 1225, dtype: float64
0       0.396939
1       0.203741
2       0.281857
3       0.426175
4       0.261323
          ...   
1220    0.466744
1221    0.486190
1222    0.365997
1223    0.162984
1224    0.515359
Name: proportion_distance_flux_jaunes, Length: 1225, dtype: float64
0       0.374408
1       0.546156
2       0.415167
3       0.229939
4       0.396278
          ...   
1220    0.170314
1221    0.184002
1222    0.296777
1223    0.538807
1224    0.248707
Name: proportion_distance_flux_verts, Length: 1225, dtype: float64
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1220    1.0
1221    1.0
1222    1.0
1223    1.0
1224    1.0
Length: 1225, dtype: float64


In [26]:
# Etape 2

# SOURCE : https://www.data.gouv.fr/fr/datasets/emission-de-co2e-par-voyageur-kilometre-sur-le-reseau/

pollution_au_km={
    "voiture":"0.097", 
    "bus":"0.092", 
    "metro":"0.0038",
    "tram":"0.0032",
    "transilien":"0.0066",
    "TER":"0.02693"
}